In [1]:
# 1. Import các thư viện cần thiết
import os
import cv2
import numpy as np
import mediapipe as mp
from pathlib import Path
import shutil
from tqdm import tqdm
import glob

print("✅ All packages imported successfully!")

✅ All packages imported successfully!


In [2]:
# 2. Khởi tạo MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Sao chép các hàm xử lý từ Action Detection Refined
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

print("✅ MediaPipe functions initialized!")

✅ MediaPipe functions initialized!


In [3]:
# 3. Hàm đổi tên file thành số tăng dần (CHUẨN SỐ LIÊN TỤC)
def rename_files_to_numbers(dataset_path):
    """
    Đổi tên tất cả file trong mỗi thư mục action thành số tăng dần từ 0: 0, 1, 2, 3, 4...
    """
    dataset_path = Path(dataset_path)
    
    if not dataset_path.exists():
        print(f"❌ Thư mục {dataset_path} không tồn tại!")
        return
    
    action_folders = [f for f in dataset_path.iterdir() if f.is_dir()]
    
    for action_folder in action_folders:
        print(f"🔄 Đang xử lý thư mục: {action_folder.name}")
        
        # Lấy tất cả file ảnh (jpg, png, jpeg)
        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
            image_files.extend(glob.glob(str(action_folder / ext)))
        
        if not image_files:
            print(f"⚠️ Không tìm thấy file ảnh nào trong {action_folder.name}")
            continue
        
        # Sắp xếp để đảm bảo thứ tự nhất quán (theo tên file)
        image_files.sort()
        
        print(f"   Tìm thấy {len(image_files)} file ảnh")
        
        # Đổi tên tạm thời để tránh xung đột (dùng .tmp)
        temp_files = []
        for i, old_path in enumerate(image_files):
            old_file = Path(old_path)
            temp_name = f"temp_rename_{i}.tmp"
            temp_path = old_file.parent / temp_name
            try:
                os.rename(old_path, temp_path)
                temp_files.append((temp_path, old_file.suffix))  # Lưu lại đuôi gốc
            except Exception as e:
                print(f"❌ Lỗi đổi tên tạm {old_path}: {e}")
        
        # Đổi tên tạm thành số liên tục: 0, 1, 2, 3...
        success_count = 0
        for idx, (temp_path, suffix) in enumerate(temp_files):
            final_name = f"{idx}{suffix}"
            final_path = temp_path.parent / final_name
            try:
                os.rename(temp_path, final_path)
                success_count += 1
            except Exception as e:
                print(f"❌ Lỗi đổi tên cuối cùng {temp_path}: {e}")
        
        print(f"✅ {action_folder.name}: Đã đổi tên thành công {success_count}/{len(image_files)} file")
        
        # Kiểm tra kết quả
        final_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
            final_files.extend(glob.glob(str(action_folder / ext)))
        final_files.sort(key=lambda x: int(Path(x).stem))
        print(f"   Kết quả: {[Path(f).name for f in final_files[:10]]}...")  # Hiển thị 10 file đầu
    
    print("🎉 Hoàn thành đổi tên tất cả file thành số liên tục!")

# Chạy hàm đổi tên với version mới
DATASET_PATH = "asl_dataset"
rename_files_to_numbers(DATASET_PATH)

🔄 Đang xử lý thư mục: 0
   Tìm thấy 560 file ảnh
❌ Lỗi đổi tên tạm asl_dataset\0\0.jpeg: [WinError 2] The system cannot find the file specified: 'asl_dataset\\0\\0.jpeg' -> 'asl_dataset\\0\\temp_rename_1.tmp'
❌ Lỗi đổi tên tạm asl_dataset\0\1.jpeg: [WinError 2] The system cannot find the file specified: 'asl_dataset\\0\\1.jpeg' -> 'asl_dataset\\0\\temp_rename_3.tmp'
❌ Lỗi đổi tên tạm asl_dataset\0\10.jpeg: [WinError 2] The system cannot find the file specified: 'asl_dataset\\0\\10.jpeg' -> 'asl_dataset\\0\\temp_rename_5.tmp'
❌ Lỗi đổi tên tạm asl_dataset\0\100.jpeg: [WinError 2] The system cannot find the file specified: 'asl_dataset\\0\\100.jpeg' -> 'asl_dataset\\0\\temp_rename_7.tmp'
❌ Lỗi đổi tên tạm asl_dataset\0\101.jpeg: [WinError 2] The system cannot find the file specified: 'asl_dataset\\0\\101.jpeg' -> 'asl_dataset\\0\\temp_rename_9.tmp'
❌ Lỗi đổi tên tạm asl_dataset\0\102.jpeg: [WinError 2] The system cannot find the file specified: 'asl_dataset\\0\\102.jpeg' -> 'asl_dataset\

In [4]:
# 4. Hàm chuyển đổi ảnh sang .npy
def convert_images_to_npy(source_path, output_path):
    """
    Chuyển đổi ảnh từ source_path sang file .npy trong output_path
    với cùng cấu trúc thư mục
    """
    source_path = Path(source_path)
    output_path = Path(output_path)
    
    # Tạo thư mục output nếu chưa có
    output_path.mkdir(exist_ok=True)
    
    # Lấy tất cả thư mục action
    action_folders = [f for f in source_path.iterdir() if f.is_dir()]
    
    total_processed = 0
    total_failed = 0
    
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as holistic:
        
        for action_folder in action_folders:
            action_name = action_folder.name
            print(f"\n🔄 Đang xử lý action: {action_name}")
            
            # Tạo thư mục output cho action này
            action_output_path = output_path / action_name
            action_output_path.mkdir(exist_ok=True)
            
            # Lấy tất cả file ảnh
            image_files = []
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                image_files.extend(glob.glob(str(action_folder / ext)))
            
            image_files.sort(key=lambda x: int(Path(x).stem))  # Sắp xếp theo số
            
            processed_count = 0
            failed_count = 0
            
            for image_path in tqdm(image_files, desc=f"Processing {action_name}"):
                try:
                    # Đọc ảnh
                    image = cv2.imread(image_path)
                    if image is None:
                        print(f"❌ Không thể đọc ảnh: {image_path}")
                        failed_count += 1
                        continue
                    
                    # Xử lý MediaPipe
                    image_rgb, results = mediapipe_detection(image, holistic)
                    
                    # Trích xuất keypoints
                    keypoints = extract_keypoints(results)
                    
                    # Tạo tên file .npy (giữ nguyên số thứ tự)
                    image_name = Path(image_path).stem
                    npy_path = action_output_path / f"{image_name}.npy"
                    
                    # Lưu file .npy
                    np.save(npy_path, keypoints)
                    processed_count += 1
                    
                except Exception as e:
                    print(f"❌ Lỗi xử lý {image_path}: {str(e)}")
                    failed_count += 1
            
            print(f"✅ {action_name}: {processed_count} thành công, {failed_count} thất bại")
            total_processed += processed_count
            total_failed += failed_count
    
    print(f"\n🎉 Hoàn thành! Tổng cộng: {total_processed} thành công, {total_failed} thất bại")
    return total_processed, total_failed

# Chạy chuyển đổi
SOURCE_PATH = "asl_dataset"
OUTPUT_PATH = "asl_dataset_npy"

processed, failed = convert_images_to_npy(SOURCE_PATH, OUTPUT_PATH)


🔄 Đang xử lý action: 0


Processing 0: 100%|██████████| 560/560 [00:19<00:00, 29.13it/s]


✅ 0: 560 thành công, 0 thất bại

🔄 Đang xử lý action: 1


Processing 1: 100%|██████████| 560/560 [00:19<00:00, 28.87it/s]


✅ 1: 560 thành công, 0 thất bại

🔄 Đang xử lý action: 2


...

✅ x: 560 thành công, 0 thất bại

🔄 Đang xử lý action: y


Processing y: 100%|██████████| 560/560 [00:17<00:00, 32.46it/s]


✅ y: 560 thành công, 0 thất bại

🔄 Đang xử lý action: z


Processing z: 100%|██████████| 560/560 [00:16<00:00, 34.14it/s]

✅ z: 560 thành công, 0 thất bại

🎉 Hoàn thành! Tổng cộng: 20120 thành công, 0 thất bại


In [8]:
# 5. Kiểm tra kết quả
def check_conversion_results(source_path, output_path):
    """
    Kiểm tra kết quả chuyển đổi
    """
    source_path = Path(source_path)
    output_path = Path(output_path)
    
    print("📊 Báo cáo kết quả chuyển đổi:")
    print("-" * 50)
    
    for action_folder in source_path.iterdir():
        if not action_folder.is_dir():
            continue
            
        action_name = action_folder.name
        
        # Đếm file ảnh gốc
        image_count = 0
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
            image_count += len(glob.glob(str(action_folder / ext)))
        
        # Đếm file .npy
        npy_folder = output_path / action_name
        npy_count = len(list(npy_folder.glob('*.npy'))) if npy_folder.exists() else 0
        
        # Hiển thị kết quả
        status = "✅" if image_count == npy_count else "⚠️"
        print(f"{status} {action_name:15} | Ảnh: {image_count:3d} | NPY: {npy_count:3d}")
    
    print("-" * 50)

check_conversion_results(SOURCE_PATH, OUTPUT_PATH)

📊 Báo cáo kết quả chuyển đổi:
--------------------------------------------------
⚠️ 0               | Ảnh: 560 | NPY: 280
⚠️ 1               | Ảnh: 560 | NPY: 280
⚠️ 2               | Ảnh: 560 | NPY: 280
⚠️ 3               | Ảnh: 560 | NPY: 280
⚠️ 4               | Ảnh: 560 | NPY: 280
⚠️ 5               | Ảnh: 560 | NPY: 280
⚠️ 6               | Ảnh: 560 | NPY: 280
⚠️ 7               | Ảnh: 560 | NPY: 280
⚠️ 8               | Ảnh: 560 | NPY: 280
⚠️ 9               | Ảnh: 560 | NPY: 280
⚠️ a               | Ảnh: 560 | NPY: 280
⚠️ b               | Ảnh: 560 | NPY: 280
⚠️ c               | Ảnh: 560 | NPY: 280
⚠️ d               | Ảnh: 560 | NPY: 280
⚠️ e               | Ảnh: 560 | NPY: 280
⚠️ f               | Ảnh: 560 | NPY: 280
⚠️ g               | Ảnh: 560 | NPY: 280
⚠️ h               | Ảnh: 560 | NPY: 280
⚠️ i               | Ảnh: 560 | NPY: 280
⚠️ j               | Ảnh: 560 | NPY: 280
⚠️ k               | Ảnh: 560 | NPY: 280
⚠️ l               | Ảnh: 560 | NPY: 280
⚠️ m             

In [6]:
# 6. Test load một file .npy để kiểm tra
def test_npy_file(npy_path):
    """
    Test load một file .npy để kiểm tra tính toàn vẹn
    """
    try:
        data = np.load(npy_path)
        print(f"✅ File: {npy_path}")
        print(f"   Shape: {data.shape}")
        print(f"   Type: {data.dtype}")
        print(f"   Sample values: {data[:5]}")
        print(f"   Has NaN: {np.isnan(data).any()}")
        print(f"   Has Inf: {np.isinf(data).any()}")
        return True
    except Exception as e:
        print(f"❌ Lỗi load file {npy_path}: {str(e)}")
        return False

# Test một file từ mỗi action
output_path = Path(OUTPUT_PATH)
for action_folder in output_path.iterdir():
    if action_folder.is_dir():
        npy_files = list(action_folder.glob('*.npy'))
        if npy_files:
            test_npy_file(npy_files[0])
            break

✅ File: asl_dataset_npy\0\0.npy
   Shape: (1662,)
   Type: float64
   Sample values: [ 0.53708756  0.25286505 -0.14097157  0.9973911   0.52093697]
   Has NaN: False
   Has Inf: False


In [7]:
# 7. Hàm tạo dataset cho training (tương tự như MP_Data)
def create_training_dataset(npy_source_path, training_output_path, sequence_length=100):
    """
    Tạo dataset training từ các file .npy đơn lẻ
    Nhóm chúng thành các sequence có độ dài cố định
    """
    npy_source_path = Path(npy_source_path)
    training_output_path = Path(training_output_path)
    training_output_path.mkdir(exist_ok=True)
    
    for action_folder in npy_source_path.iterdir():
        if not action_folder.is_dir():
            continue
            
        action_name = action_folder.name
        print(f"🔄 Tạo sequences cho {action_name}")
        
        # Tạo thư mục cho action
        action_training_path = training_output_path / action_name
        action_training_path.mkdir(exist_ok=True)
        
        # Lấy tất cả file .npy và sắp xếp
        npy_files = sorted(action_folder.glob('*.npy'), 
                          key=lambda x: int(x.stem))
        
        if len(npy_files) < sequence_length:
            print(f"⚠️ {action_name} chỉ có {len(npy_files)} file, ít hơn {sequence_length}")
            continue
        
        # Tạo sequences
        sequence_id = 0
        for i in range(0, len(npy_files) - sequence_length + 1, sequence_length):
            sequence_folder = action_training_path / str(sequence_id)
            sequence_folder.mkdir(exist_ok=True)
            
            # Copy 100 file liên tiếp vào sequence
            for j in range(sequence_length):
                source_file = npy_files[i + j]
                target_file = sequence_folder / f"{j}.npy"
                shutil.copy2(source_file, target_file)
            
            sequence_id += 1
        
        print(f"✅ {action_name}: Tạo được {sequence_id} sequences")

# Tạo dataset training
TRAINING_OUTPUT_PATH = "MP_Data_from_images"
create_training_dataset(OUTPUT_PATH, TRAINING_OUTPUT_PATH, sequence_length=100)

🔄 Tạo sequences cho 0
✅ 0: Tạo được 2 sequences
🔄 Tạo sequences cho 1
✅ 1: Tạo được 2 sequences
🔄 Tạo sequences cho 2
✅ 2: Tạo được 2 sequences
🔄 Tạo sequences cho 3
✅ 3: Tạo được 2 sequences
🔄 Tạo sequences cho 4
✅ 4: Tạo được 2 sequences
🔄 Tạo sequences cho 5
✅ 5: Tạo được 2 sequences
🔄 Tạo sequences cho 6
✅ 6: Tạo được 2 sequences
🔄 Tạo sequences cho 7
✅ 7: Tạo được 2 sequences
🔄 Tạo sequences cho 8
✅ 8: Tạo được 2 sequences
🔄 Tạo sequences cho 9
✅ 9: Tạo được 2 sequences
🔄 Tạo sequences cho a
✅ a: Tạo được 2 sequences
🔄 Tạo sequences cho b
✅ b: Tạo được 2 sequences
🔄 Tạo sequences cho c
✅ c: Tạo được 2 sequences
🔄 Tạo sequences cho d
✅ d: Tạo được 2 sequences
🔄 Tạo sequences cho e
✅ e: Tạo được 2 sequences
🔄 Tạo sequences cho f
✅ f: Tạo được 2 sequences
🔄 Tạo sequences cho g
✅ g: Tạo được 2 sequences
🔄 Tạo sequences cho h
✅ h: Tạo được 2 sequences
🔄 Tạo sequences cho i
✅ i: Tạo được 2 sequences
🔄 Tạo sequences cho j
✅ j: Tạo được 2 sequences
🔄 Tạo sequences cho k
✅ k: Tạo được 2 se